In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from Utilities.get_image_data import load_data
from Utilities.form_train_val_test_batches import form_train_val_test_batches
from Utilities.NN_FC_layerwise import FCLayerwise
from Utilities.loss_and_accuracies import data_loss_classification, accuracy_classification
from Utilities.manifold_regularization import manifold_classification
from Utilities.optimize_layerwise import optimize
from Utilities.plot_and_save_figures_layerwise import plot_fig
from joblib import Parallel, delayed
import multiprocessing
import random

In [3]:
from decimal import Decimal # for filenames

import pdb #Equivalent of keyboard in MATLAB, just add "pdb.set_trace()"

import os
import sys

In [4]:
###############################################################################
#                       HyperParameters and RunOptions                        #
###############################################################################
class Hyperparameters:
    max_hidden_layers = 4 # For this architecture, need at least 2. One for the mapping to the feature space, one as a trainable hidden layer. EXCLUDES MAPPING BACK TO DATA SPACE
    num_hidden_nodes  = 10
    activation        = 'elu'
    regularization    = 0.001
    manifold          = 0.006
    node_TOL          = 1e-4
    error_TOL         = 1e-4
    batch_size        = 1000
    num_epochs        = 50
    

In [5]:
class RunOptions:
    def __init__(self):    
        #=== Choose Which GPU to Use ===#
        self.which_gpu = '1'
        
        #=== Use L_1 Regularization ===#
        self.use_L1 = 1
        
        #=== Choose Data Set ===#
        self.data_MNIST = 1
        self.data_CIFAR10 = 0 
        self.data_CIFAR100 = 0
        
        #=== Random Seed ===#
        self.random_seed = 1234


In [6]:
###############################################################################
#                                 File Paths                                  #
###############################################################################         
class FilePaths():    
    def __init__(self, hyperp, run_options):  
        #=== Declaring File Name Components ===# 
        self.NN_type = 'FC'
        if run_options.data_MNIST == 1:
            self.dataset = 'MNIST'
        if run_options.data_CIFAR10 == 1:
            self.dataset = 'CIFAR10'
        if run_options.data_CIFAR100 == 1:
            self.dataset = 'CIFAR100'
        if hyperp.regularization >= 1:
            hyperp.regularization = int(hyperp.regularization)
            regularization_string = str(hyperp.regularization)
        else:
            regularization_string = str(hyperp.regularization)
            regularization_string = 'pt' + regularization_string[2:]                        
        node_TOL_string = str('%.2e' %Decimal(hyperp.node_TOL))
        node_TOL_string = node_TOL_string[-1]
        error_TOL_string = str('%.2e' %Decimal(hyperp.error_TOL))
        error_TOL_string = error_TOL_string[-1]
        
        #=== File Name ===#
        if run_options.use_L1 == 0:
            self.filename = self.dataset + '_' + self.NN_type + '_mhl%d_hl%d_eTOL%s_b%d_e%d' %(hyperp.max_hidden_layers, hyperp.num_hidden_nodes, error_TOL_string, hyperp.batch_size, hyperp.num_epochs)
        else:
            self.filename = self.dataset + '_' + self.NN_type + '_L1_mhl%d_hl%d_r%s_nTOL%s_eTOL%s_b%d_e%d' %(hyperp.max_hidden_layers, hyperp.num_hidden_nodes, regularization_string, node_TOL_string, error_TOL_string, hyperp.batch_size, hyperp.num_epochs)

        #=== Saving Trained Neural Network and Tensorboard ===#
        #self.NN_savefile_directory = 'C:/Users/Chandradut/Desktop/Sparse training/Trained_NNs/' + self.filename # Since we need to save four different types of files to save a neural network model, we need to create a new folder for each model
        self.NN_savefile_directory =  self.filename
        self.NN_savefile_name = self.NN_savefile_directory + '/' + self.filename # The file path and name for the four files
        #self.tensorboard_directory = 'C:/Users/Chandradut/Desktop/Sparse training/Tensorboard/' + self.filename

###############################################################################
#                                 Training                                    #
###############################################################################
def trainer(hyperp, run_options, file_paths):
    #=== GPU Settings ===#
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
    os.environ["CUDA_VISIBLE_DEVICES"] = run_options.which_gpu
    
    #=== Load Data ===#       
    data_train, labels_train,\
    data_test, labels_test,\
    data_input_shape, num_channels, label_dimensions\
    = load_data(file_paths.NN_type, file_paths.dataset, run_options.random_seed) 
    
    #=== Construct Validation Set and Batches ===# 
    data_and_labels_train, data_and_labels_val, data_and_labels_test,\
    num_data_train, num_data_val, num_data_test,\
    num_batches_train, num_batches_val, num_batches_test,data_and_labels_train_new\
    = form_train_val_test_batches(data_train, labels_train, \
                                  data_test, labels_test, \
                                  hyperp.batch_size, run_options.random_seed)
    
    #=== Neural Network ===#
    if run_options.use_L1 == 0:
        kernel_regularizer = None
        bias_regularizer = None  
    else:
        kernel_regularizer = tf.keras.regularizers.l1(hyperp.regularization)
        bias_regularizer = tf.keras.regularizers.l1(hyperp.regularization)
    NN = FCLayerwise(hyperp, run_options, data_input_shape, label_dimensions,
                      kernel_regularizer, bias_regularizer)    
    
    #=== Training ===#
    optimize(hyperp, run_options, file_paths, NN, data_loss_classification, accuracy_classification, data_and_labels_train, data_and_labels_val, data_and_labels_test, label_dimensions, num_batches_train,data_and_labels_train_new,manifold_classification,hyperp.batch_size,run_options.random_seed,num_data_train)
    
###############################################################################
#                                 Driver                                      #
###############################################################################     

In [7]:
if __name__ == "__main__":     

    #=== Hyperparameters and Run Options ===#    
    hyperp = Hyperparameters()
    run_options = RunOptions()
    
    #if len(sys.argv) > 1:
        #hyperp.max_hidden_layers = int(sys.argv[1])
        #hyperp.num_hidden_nodes  = int(sys.argv[2])
        #hyperp.activation        = str(sys.argv[3])
        #hyperp.regularization    = float(sys.argv[4])
        #hyperp.node_TOL          = float(sys.argv[5])
        #hyperp.error_TOL         = float(sys.argv[6])
        #hyperp.batch_size        = int(sys.argv[7])
        #hyperp.num_epochs        = int(sys.argv[8])
        #run_options.which_gpu    = int(sys.argv[9])
            
    #=== File Names ===#
    file_paths = FilePaths(hyperp, run_options)
    
    #=== Initiate training ===#
    trainer(hyperp, run_options, file_paths) 
    plot_fig(hyperp, run_options, file_paths)

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
ListWrapper([784, 10, 10, 10])
Initial Losses:
Training Set: Loss: 2.297e+00, Accuracy: 0.150
Validation Set: Loss: 2.613e+00, Accuracy: 0.151
Test Set: Loss: 2.613e+00, Accuracy: 0.157

Beginning Training
            Epoch 0            
MNIST_FC_L1_mhl4_hl10_rpt001_nTOL4_eTOL4_b1000_e50
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 48 batches of size 1000:
Model: "fc_layerwise"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
W2 (Dense)                   multiple                  110       
_________________________________________________________________
upsampling_layer (Dense)     multiple                  7850      
_________________________________________________________________
classification_layer (Dense) multiple                  110       
Total params: 8,070
Trainable params: 8,070
Non-trainable params: 0
_____

Time per Batch: 0.06
Time per Epoch: 4.33

Training Set: Loss: 2.296e+00, Accuracy: 0.112
Validation Set: Loss: 2.364e+00, Accuracy: 0.110
Test Set: Loss: 2.364e+00, Accuracy: 0.113

Previous Layer Relative # of 0s: 0.0000000

            Epoch 16            
MNIST_FC_L1_mhl4_hl10_rpt001_nTOL4_eTOL4_b1000_e50
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 48 batches of size 1000:
Time per Batch: 0.06
Time per Epoch: 4.51

Training Set: Loss: 2.296e+00, Accuracy: 0.112
Validation Set: Loss: 2.356e+00, Accuracy: 0.110
Test Set: Loss: 2.356e+00, Accuracy: 0.113

Previous Layer Relative # of 0s: 0.0000000

            Epoch 17            
MNIST_FC_L1_mhl4_hl10_rpt001_nTOL4_eTOL4_b1000_e50
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 48 batches of size 1000:
Time per Batch: 0.06
Time per Epoch: 4.50

Training Set: Loss: 2.295e+00, Accuracy: 0.112
Validation Set: Loss: 2.349e+00, Accuracy: 0.112
Test Set: Loss: 2.350e+00, Accuracy: 0.113

Previous Layer Relative # of 0s: 0.0000000


Time per Batch: 0.07
Time per Epoch: 4.49

Training Set: Loss: 2.169e+00, Accuracy: 0.313
Validation Set: Loss: 2.181e+00, Accuracy: 0.345
Test Set: Loss: 2.181e+00, Accuracy: 0.342

Previous Layer Relative # of 0s: 0.0000000

            Epoch 35            
MNIST_FC_L1_mhl4_hl10_rpt001_nTOL4_eTOL4_b1000_e50
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 48 batches of size 1000:
Time per Batch: 0.06
Time per Epoch: 4.21

Training Set: Loss: 2.145e+00, Accuracy: 0.345
Validation Set: Loss: 2.165e+00, Accuracy: 0.315
Test Set: Loss: 2.163e+00, Accuracy: 0.317

Previous Layer Relative # of 0s: 0.0000000

            Epoch 36            
MNIST_FC_L1_mhl4_hl10_rpt001_nTOL4_eTOL4_b1000_e50
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 48 batches of size 1000:
Time per Batch: 0.06
Time per Epoch: 4.31

Training Set: Loss: 2.124e+00, Accuracy: 0.370
Validation Set: Loss: 2.146e+00, Accuracy: 0.361
Test Set: Loss: 2.142e+00, Accuracy: 0.371

Previous Layer Relative # of 0s: 0.0000000


Time per Epoch: 4.56

Training Set: Loss: 1.637e+00, Accuracy: 0.580
Validation Set: Loss: 1.676e+00, Accuracy: 0.584
Test Set: Loss: 1.674e+00, Accuracy: 0.579

Previous Layer Relative # of 0s: 0.1363636

            Epoch 1            
MNIST_FC_L1_mhl4_hl10_rpt001_nTOL4_eTOL4_b1000_e50
Trainable Hidden Layer Index: 3
GPU: 1

Optimizing 48 batches of size 1000:
Time per Batch: 0.06
Time per Epoch: 4.48

Training Set: Loss: 1.488e+00, Accuracy: 0.613
Validation Set: Loss: 1.528e+00, Accuracy: 0.620
Test Set: Loss: 1.527e+00, Accuracy: 0.614

Previous Layer Relative # of 0s: 0.1363636

            Epoch 2            
MNIST_FC_L1_mhl4_hl10_rpt001_nTOL4_eTOL4_b1000_e50
Trainable Hidden Layer Index: 3
GPU: 1

Optimizing 48 batches of size 1000:
Time per Batch: 0.07
Time per Epoch: 4.59

Training Set: Loss: 1.450e+00, Accuracy: 0.627
Validation Set: Loss: 1.506e+00, Accuracy: 0.640
Test Set: Loss: 1.498e+00, Accuracy: 0.643

Previous Layer Relative # of 0s: 0.1363636

            Epoch 3   

KeyboardInterrupt: 

In [10]:
import random
random.randint(1,5)

3

In [8]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import datasets
kernel_regularizer = tf.keras.regularizers.l1(hyperp.regularization)
bias_regularizer = tf.keras.regularizers.l1(hyperp.regularization)
(data_train, labels_train), (data_test, labels_test) = datasets.mnist.load_data()
data_train = data_train.reshape(data_train.shape[0], 28, 28, 1)
data_test = data_test.reshape(data_test.shape[0], 28, 28, 1)
label_dimensions = 10
data_input_shape = data_train.shape[1:]

In [9]:
NN = FCLayerwise(hyperp, run_options, data_input_shape, label_dimensions,
                      kernel_regularizer, bias_regularizer)  

ListWrapper([28, 10, 10, 10])


In [10]:
(data_train, labels_train), (data_test, labels_test) = datasets.mnist.load_data()
data_train = data_train.reshape(data_train.shape[0], 28, 28, 1)
data_test = data_test.reshape(data_test.shape[0], 28, 28, 1)
label_dimensions = 10
data_input_shape = data_train.shape[1:]

#=== Casting as float32 ===#
data_train = tf.cast(data_train,tf.float32)
labels_train = tf.cast(labels_train, tf.int32)
data_test = tf.cast(data_test, tf.float32)
labels_test = tf.cast(labels_test, tf.int32)
    
#=== Normalize Data ===#
data_train, data_test = data_train/255.0, data_test/255.0
data_train = tf.image.per_image_standardization(data_train) # Linearly scales each image to have mean 0 and variance 1
data_test = tf.image.per_image_standardization(data_test)   # Linearly scales each image to have mean 0 and variance 1

data_train = tf.reshape(data_train, (len(data_train), 28*28))
data_test = tf.reshape(data_test, (len(data_test), 28*28))

num_data_train = len(data_train)
num_data_test = len(data_test)
random_seed=1234
batch_size        = 1000

data_and_labels_train_full = tf.data.Dataset.from_tensor_slices((data_train, labels_train)).shuffle(num_data_train, seed=random_seed)
data_and_labels_test = tf.data.Dataset.from_tensor_slices((data_test, labels_test)).batch(batch_size)
num_batches_test = len(list(data_and_labels_test))

#=== Partitioning Out Validation Set and Constructing Batches ===#
current_num_data_train = num_data_train
num_data_train = int(0.8 * num_data_train)
num_data_val = current_num_data_train - num_data_train
data_and_labels_train_full = tf.data.Dataset.from_tensor_slices((data_train, labels_train)).shuffle(num_data_train, seed=random_seed)
data_and_labels_train = data_and_labels_train_full.take(num_data_train).batch(batch_size)
data_and_labels_val = data_and_labels_train_full.skip(num_data_train).batch(batch_size)    
num_batches_train = len(list(data_and_labels_train))
num_batches_val = len(list(data_and_labels_val))

In [11]:
for batch_num, (batch_data_train, batch_labels_train) in data_and_labels_train.enumerate():
    batch=batch_num
    batch_data_train = batch_data_train
    batch_labels_train=batch_labels_train

In [12]:
x_train_new = batch_data_train[batch_labels_train == 0]
batch_pred_train,val=NN(x_train_new)
dimension=np.shape(val)

In [19]:
new_one=val[1:dimension[0]]

In [37]:
np.mean(tf.keras.losses.mean_squared_error(new_one, val[0:dimension[0]-1]))

1.9269863

array([4, 6, 6, ..., 7, 2, 5])

In [19]:

x_train_new = batch_data_train[batch_labels_train == 1]
batch_pred_train,val=NN(x_train_new)



In [16]:
y_true = tf.one_hot(tf.cast(y_true,tf.int64), label_dimensions, dtype=tf.float32)

In [45]:
loss

<tf.Tensor: shape=(), dtype=float32, numpy=1.6097095>

In [48]:
np.linalg.norm(val[0]-val[1])

2.567157